## Product-type(category) task: Multi-label classification 

Author: Nana

Date: 2022/4/14


In [1]:
from google.colab import drive
drive.mount('/content/drive')
maindir = '/content/drive/MyDrive/FinTech-final-project'
datadir = f'{maindir}/data'
spmdir = f'{maindir}/spm'
modeldir = f'{maindir}/models'
cat_df_path = f'{maindir}/東吳課程_發票資料集/品類資料集/cat_train_v2.csv'

Mounted at /content/drive


## Classifying fasttext embeddings
MLknn
https://notebook.community/scikit-multilearn/scikit-multilearn/docs/source/multilabelembeddings

In [7]:
!pip -q install scikit-multilearn

     |████████████████████████████████| 89 kB 5.1 MB/s 


In [8]:
# openNE is required in skmultilearn
!pip -q install networkx tensorflow
!git clone https://github.com/thunlp/OpenNE/
!pip -q install -e OpenNE/src

     |████████████████████████████████| 462 kB 12.6 MB/s 
Cloning into 'OpenNE'...
remote: Enumerating objects: 2700, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 2700 (delta 5), reused 9 (delta 3), pack-reused 2677
Receiving objects: 100% (2700/2700), 116.28 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (1564/1564), done.


In [9]:
import joblib, sys

In [ ]:
# required for skmultilearn (can skip if not using the package)
# # follow the instructions on https://github.com/thunlp/OpenNE
!pip install -r requirements.txt
%cd OpenNE/src
!python setup.py install

## Preparing Data (X, y) 

http://scikit.ml/multilabeldnn.html#Multi-label-deep-learning-with-scikit-multilearn



In [2]:
fmodelpath = f'{modeldir}/fasttext.model'
ypath = f'{datadir}/category/category_labels.pkl'

In [17]:
import joblib 
from collections import defaultdict
LabelsList = joblib.load(ypath)

In [18]:
CatsDict = defaultdict(list)
for x in LabelsList:
  # print(x.keys())
  for cat in x['product']:
    # print(x['name'])
    CatsDict[cat].append(x['name'])

In [21]:
import random
print('# of categories:', len(CatsDict.keys()))
for k, v in CatsDict.items():
  print(f"Category: {k}, Length: {len(v)}, \nExamples:{random.sample(v, min(10, len(v)))}")
  print('---------------')
  # break

# of categories: 217
Category: 人工淚液, Length: 121, 
Examples:['R佐藤視朗點眼液15ml', 'R參天柔潤人工淚液點眼液5ml-4入', '61213', '(R)新鮮視OA點眼液', 'Sato 視朗眼液15', '愛力根優麗舒(R)單支裝眼用點眼液', 'C-SINOMIN止膿敏點眼液15ML(裸)', 'R參天柔潤人工淚液點眼液 5ml 4入', 'R視舒坦人工淚液點眼液10ml', '參天柔潤人工淚液點眼液5ml-4入']
---------------


In [23]:
import json
with open(f"{datadir}/category/category_dictionary.json", "w") as outfile:
    json.dump(CatsDict, outfile, ensure_ascii = False, indent = 4)

In [60]:
# average the token vectors to form X's feature vector
# use the binary tuple as label
from gensim.models import FastText
# should save keyedvectors only if there's no further training 
model = FastText.load(fmodelpath) 

In [61]:
## segmented, should have saved the segmented results for this
# !pip install -q -U ckip-transformers
# from ckip_transformers.nlp import CkipWordSegmenter
# ws_driver = CkipWordSegmenter(level=3, device = 0)
# name_texts = [x['name'] for x in LabelsList]
# ws = ws_driver(name_texts) !nvidia-smi # k80要20分鐘

In [63]:
import numpy as np
# parameters
emb_dim = 200 # (see notebook 12.0)
Xlen = len(LabelsList)
nclass = len(LabelsList[0]['labels'])

# make X(Emb matrix), y (label matrix)
Emb = np.zeros((Xlen, emb_dim))
Labels = np.zeros((Xlen, nclass))

def get_avg_embeddings(tokenlist):
  # if no token has embedding available, yields np.zeros(emb_dim)
  return np.mean([model.wv[tok] if tok in model.wv else np.zeros(emb_dim) for tok in tokenlist], axis = 0)

def get_weighted_embeddings(tokenlist):
  # ref: https://stackoverflow.com/questions/29330792/weighted-averaging-a-list
  rate = np.ones(len(tokenlist))
  mid = int(len(rate)/2)
  # category name通常出現在產品名稱後半部
  rate = np.append(rate[:mid], (rate[mid:]+1))
  # print(rate)
  X = [model.wv[tok] if tok in model.wv else np.zeros(emb_dim) for tok in tokenlist]
  return np.average(X, 
                    axis = 0,
                    weights=rate)

In [64]:
for i in range(Xlen):
    Emb[i,:] = get_weighted_embeddings(LabelsList[i]['seg_name'])
    Labels[i,:] = LabelsList[i]['labels']

In [ ]:
get_weighted_embeddings(['黑人','牙膏'])

In [ ]:
print(model.wv['黑人'])
print(model.wv['牙膏'])

## Training
stack a simple few-layer neural network 

In [50]:
!pip install -q -U torch

     |████████████████████████████████| 750.6 MB 12 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.


In [51]:
# seeding; normally not working in colab
import torch
from torch import nn
import torch.nn.functional as F
myseed = 42
torch.manual_seed(myseed)
torch.cuda.manual_seed(myseed)
torch.cuda.manual_seed_all(myseed)
np.random.seed(myseed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [86]:
from sklearn.model_selection import train_test_split
# 一些分工：做k-fold、
X_train, X_test, y_train, y_test = train_test_split(Emb, Labels, test_size=0.2, 
                                                    random_state=myseed)
input_dim = X_train.shape[1]
hd1, hd2 = 128, 256
dropout = 0.5
output_dim = nclass # originally len(np.unique(y_train.rows)) as http://scikit.ml/multilabeldnn.html#Multi-label-deep-learning-with-scikit-multilearn 
# states, but the logic is (if I understand it correctly) very weird

In [90]:
from torch.utils.data import Dataset

class CatDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.from_numpy(y).float()
        else:
            self.label = None
    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]
    def __len__(self):
        return len(self.data)


In [91]:
from torch.utils.data import DataLoader
batch_size = 200
train_set = CatDataset(X_train, y_train)
val_set = CatDataset(X_test, y_test)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

## Architecture(s): Simple Network (linear layers)


In [96]:
class MultiLabelClassifier(nn.Module):
    def __init__(
            self,
            input_dim=input_dim,
            hd1 = hd1,
            hd2 = hd2,
            output_dim=output_dim,
            dropout=dropout,
    ):
        super(MultiLabelClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 只有兩層fc layers (shallow)，主要是看fasttext embedding的效果
        self.hidden1 = nn.Linear(input_dim, hd1)
        self.hidden2 = nn.Linear(hd1, hd2)
        self.output = nn.Linear(hd2, output_dim)

    def forward(self, X, **kwargs):
        X = self.hidden1(X)
        X = self.hidden2(X)
        X = F.relu(X)
        X = self.dropout(X)
        # 過softmax拿機率值
        X = F.softmax(self.output(X), dim=-1)
        return X

References: 
https://keras.io/examples/nlp/pretrained_word_embeddings/
https://towardsdatascience.com/

lstm-text-classification-using-pytorch-2c6c657f8fc0

My machine learning assignments

In [97]:
# from skmultilearn.problem_transform import LabelPowerset
# training parameters
THRESHOLD = 0.3
epochs = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 5e-4       # learning rate
net = MultiLabelClassifier().to(device)
criterion = nn.BCELoss() 
weight_decay = 0.1
optimizer = torch.optim.AdamW(net.parameters(), 
                              lr=learning_rate, 
                              weight_decay=weight_decay)

In [110]:
# simple version: Macro f1: 0.2 ?? 

In [56]:
import warnings
warnings.filterwarnings("ignore")

In [98]:
# Training Loop
from sklearn.metrics import f1_score
best_acc = 0.0
for epoch in range(epochs):
    train_acc, val_acc = 0.0, 0.0
    train_loss, val_loss = 0.0,0.0
    # training
    net.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        # print('l:', labels.shape)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = net(inputs) 
        # print('o:', outputs.shape, 'l:', labels.shape)
        batch_loss = criterion(outputs, labels)
        
        # THRESHOLD: all labels with probabilities higher than it are considered predicted labels 
        # and others are skipped. We are using a threshold value of 0.5.
        pred = np.array(outputs.cpu().detach().numpy() > THRESHOLD, dtype=float)
        labels = labels.cpu().detach().numpy()
        train_acc += f1_score(labels, pred, average='macro')
        # print(f1_score(labels, pred, average='macro'))
        # compute macro_f1
        # print(labels.shape, pred.shape)
        # print(f1_score(labels, pred, average='macro'))
        batch_loss.backward() 
        optimizer.step() 
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        net.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs =net(inputs)
                batch_loss = criterion(outputs, labels) 
                val_pred = np.array(outputs.cpu().detach().numpy() > THRESHOLD, dtype=float)
                labels = labels.cpu().detach().numpy()
                val_acc += f1_score(labels, val_pred, average='macro')
                # compute macro_f1
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Macro f1: {:3.6f} Loss: {:3.6f} | Val Macro f1: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, epochs, train_acc/len(train_loader), train_loss/len(train_loader), val_acc/len(val_loader), val_loss/len(val_loader)
            ))

[001/100] Train Macro f1: 0.072467 Loss: 0.016402 | Val Macro f1: 0.147684 loss: 0.010557
[002/100] Train Macro f1: 0.161240 Loss: 0.010588 | Val Macro f1: 0.197878 loss: 0.008691
[003/100] Train Macro f1: 0.195289 Loss: 0.009274 | Val Macro f1: 0.223527 loss: 0.007875
[004/100] Train Macro f1: 0.214887 Loss: 0.008524 | Val Macro f1: 0.238000 loss: 0.007356
[005/100] Train Macro f1: 0.226800 Loss: 0.008064 | Val Macro f1: 0.248521 loss: 0.007014
[006/100] Train Macro f1: 0.237154 Loss: 0.007720 | Val Macro f1: 0.255675 loss: 0.006745
[007/100] Train Macro f1: 0.242494 Loss: 0.007467 | Val Macro f1: 0.261538 loss: 0.006518
[008/100] Train Macro f1: 0.250259 Loss: 0.007226 | Val Macro f1: 0.265354 loss: 0.006332
[009/100] Train Macro f1: 0.254694 Loss: 0.007026 | Val Macro f1: 0.269082 loss: 0.006205
[010/100] Train Macro f1: 0.259037 Loss: 0.006871 | Val Macro f1: 0.274297 loss: 0.006063
[011/100] Train Macro f1: 0.262538 Loss: 0.006741 | Val Macro f1: 0.275832 loss: 0.005957
[012/100] 

In [103]:
f1_score(labels, val_pred, average='macro')

0.1675840978593272

In [102]:
val_pred[1,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [83]:
from sklearn.metrics import f1_score
# multilabel classification
y_true = [[0, 0, 0], [1, 1, 1], [0, 1, 1]]
y_pred = [[0, 0, 0], [1, 1, 1], [1, 1, 0]]
f1_score(y_true, y_pred, average='macro')

0.7777777777777777

## Checking example reference: 
http://scikit.ml/multilabeldnn.html#Multi-label-deep-learning-with-scikit-multilearn
(which doesn't seem to have the logic our task requires)

In [35]:
!pip install arff
from skmultilearn.dataset import load_dataset
X_train, y_train, feature_names, label_names = load_dataset('emotions', 'train')
X_test, y_test, _, _ = load_dataset('emotions', 'test')

  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4971 sha256=e32881b68341e9e34382d8239db4b8531a7b17ffb35a7d841e26a72a76071a58
  Stored in directory: /root/.cache/pip/wheels/fe/81/bd/4ae90e24ba860304e375da219f9205b2586dbee255f3ee70e2
Successfully built arff
emotions:train - does not exists downloading
Downloaded emotions-train
emotions:test - does not exists downloading
Downloaded emotions-test


In [ ]:
y_train.rows # ?? probably not what I expected